<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/EM_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

Mounted at /content/drive


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [2]:
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  #print(i)
  train = train.append(i)



In [3]:
train.sort_values("Sentence", inplace = True)
print(len(train))


37711


In [4]:
 new_train = train.drop_duplicates(subset ="Sentence")


In [5]:
train = new_train

In [6]:
train['Target'].unique()

array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Others',
       'Conclusion', 'Issue'], dtype=object)

In [7]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()


array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Conclusion',
       'Issue'], dtype=object)

In [8]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
len(train['Sentence'])

4416

In [10]:
text = clean_text(train['Sentence'])

In [11]:
type(text)

list

In [12]:
len(text)

59974

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(4416, 7374)


In [14]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [15]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

unlabel_1 = unlabel.loc[:500]


def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  #print(unlabel_2.head())
  return unlabel_2

unlabel_1 = index_reset(unlabel_1)
unlabel_1_copy = unlabel_1



x_un1 = tfidf_vect.transform(unlabel_1['text'])






[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
unlabel_1['Target']=-1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
import numpy as np
np.unique(unlabel_1['Target'])

array([-1])

In [18]:
train = train.rename(columns={'Sentence':'text'})

In [19]:
 from sklearn.preprocessing import LabelEncoder
 train['Target']= LabelEncoder().fit_transform(train['Target'])

In [20]:
train_and_unlabel =  pd.concat([train,unlabel_1])
print(len(train))
print(len(unlabel_1))
print(len(train_and_unlabel))

4416
501
4917


In [21]:
train_and_unlabel.head()

,Unnamed: 0,text,Target
1364,239.0,"""(I)n the First Amendment area 'government may...",3
1833,185.0,"""... that nowhere in the statute was it stated...",3
2431,415.0,"""Although a statute may be neither vague, over...",5
2245,416.0,"""For even when pursuing a legitimate interest,...",5
1561,25.0,"""If an indictment has been found or accusation...",2


In [22]:

train_and_unlabel.reset_index(inplace=True)

In [23]:
train_and_unlabel.columns

Index(['index', 'Unnamed: 0', 'text', 'Target'], dtype='object')

In [24]:
del train_and_unlabel['index']
del train_and_unlabel['Unnamed: 0']

In [25]:
train_and_unlabel.head()

,text,Target
0,"""(I)n the First Amendment area 'government may...",3
1,"""... that nowhere in the statute was it stated...",3
2,"""Although a statute may be neither vague, over...",5
3,"""For even when pursuing a legitimate interest,...",5
4,"""If an indictment has been found or accusation...",2


In [26]:
train_and_unlabel.tail()

,text,Target
4912,time continuance partnership mr winn jack stal...,-1
4913,correspondence appellant went coleman county s...,-1
4914,appellant contention stock sold winn named pri...,-1
4915,mr winns contention stock delivered appellant ...,-1
4916,title stock passed appellant would guilty embe...,-1


In [27]:
# #cleaning
# import nltk
# import re
# import string
# nltk.download('stopwords')
# nltk.download('wordnet')
# stopword=nltk.corpus.stopwords.words('english')
# from nltk.stem import WordNetLemmatizer
# wl= WordNetLemmatizer()

# def clean_text(text):
#   text="".join([word.lower() for word in text if word not in string.punctuation])
#   tokens = re.split('\W+',text)
#   text = [wl.lemmatize(word) for word in tokens if word not in stopword]
#   return text

In [28]:
train_and_unlabel_words = clean_text(train_and_unlabel['text'])

In [29]:
len(train_and_unlabel_words)

66254

In [30]:
train_and_unlabel_words_wo_duplicates = list(set(train_and_unlabel_words))

In [31]:
len(train_and_unlabel_words_wo_duplicates)

8129

In [32]:
type(train_and_unlabel_words_wo_duplicates)

list

In [33]:
len(train_and_unlabel_words_wo_duplicates)

8129

In [34]:
train_words = clean_text(train['text'])

In [35]:
len(np.unique(train_words))

7232

In [36]:
train_words_wo_duplicates = list(set(train_words))

In [37]:
len(train_words_wo_duplicates)

7232

In [38]:
unlabel_words = clean_text(unlabel_1['text'])

In [39]:
len(unlabel_words)

6281

In [40]:
len(np.unique(unlabel_words))

2328

In [41]:
unlabel_words_wo_duplicates = list(set(unlabel_words))

In [42]:
len(unlabel_words_wo_duplicates)

2328

In [43]:
PF = []
NF = []
Threshold_feature = 1

def feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words):
  for term in train_and_unlabel_words_wo_duplicates:
    print(term)
    freq_train = train_words.count(term) / len(train_words)
    print(freq_train)
    freq_unlabel = unlabel_words.count(term) / len(unlabel_words)
    print(freq_unlabel)
    if freq_unlabel == 0:
      PF.append(term)
    elif (freq_train // freq_unlabel) > Threshold_feature:
      PF.append(term)
    else:
      NF.append(term)


feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words)
    


Streaming output truncated to the last 5000 lines.
1.667389201987528e-05
0.0
psychology
1.667389201987528e-05
0.0
convicted
0.0011838463334111447
0.001910523801942366
aria
1.667389201987528e-05
0.0
aforethought
6.669556807950112e-05
0.0
jallowed
0.0
0.0001592103168285305
answered
0.00021676059625837864
0.0
transfer
0.00021676059625837864
0.0
relating
0.000300130056357755
0.0001592103168285305
reviewed
0.00011671724413912695
0.0
caseoffense
0.0
0.0001592103168285305
excused
1.667389201987528e-05
0.0
useless
1.667389201987528e-05
0.0
assistance
3.334778403975056e-05
0.0
supp
1.667389201987528e-05
0.0
expected
6.669556807950112e-05
0.0
reyesâ
1.667389201987528e-05
0.0
zandt
1.667389201987528e-05
0.0
jobâ
1.667389201987528e-05
0.0
robinson
3.334778403975056e-05
0.0
lawoverruled
0.0
0.0001592103168285305
anything
0.0005168906526161337
0.0004776309504855915
trial
0.0052189282022209625
0.0030249960197420793
labium
1.667389201987528e-05
0.0
instead
8.33694600993764e-05
0.0
page
0.0002501083802

In [44]:
len(PF)

5959

In [45]:
len(NF)

2170

In [46]:
len(PF) + len(NF)

8129

In [47]:
# unlabel_1

In [48]:
# RN = unlabel_1[0:10]

In [49]:
# unlabel_2 = unlabel_1[0:10]

In [50]:
# RN = unlabel_2

In [51]:
# RN.drop(pos_to_be_removed,axis=0,inplace=True)

In [52]:
# pos_to_be_removed

In [ ]:


def RN_selection(RN, unlabel_1):
  iteration_RN = 0
  RN_to_be_removed = []
  pos_to_be_removed = []
  freq_each_word_list = []
  count = 0

  for doc in unlabel_1['text']:
    print("iteration_RN",iteration_RN)
    print(doc)
    doc_words = clean_text(doc)
    doc_words_wo_duplicates = list(set(doc_words))
    for each_doc_word in doc_words_wo_duplicates:
      print(each_doc_word)
      freq_each_doc_word = doc_words.count(each_doc_word)
      freq_each_word_list.append(freq_each_doc_word)
      print(freq_each_doc_word)
      print(each_doc_word in PF)
      if (freq_each_doc_word > 0) and (each_doc_word in PF):
        print("Yes writing ........")
        pos_to_be_removed.append(count)
        RN_to_be_removed.append(doc)
        print("breaking:::::::::::")
        break
    count = count + 1
    iteration_RN = iteration_RN + 1
  
  RN.drop(pos_to_be_removed,axis=0,inplace=True)
  #Q =  unlabel_1.loc[pos_to_be_removed,:]
  #print(len(Q))
  return RN,pos_to_be_removed

unlabel_2 = unlabel_1[0:10] #comment it 
RN = unlabel_2
RN,pos_to_be_removed = RN_selection(RN,unlabel_2)

In [56]:
len(RN)

3

In [58]:
pos_to_be_removed

[1, 3, 4, 5, 6, 7, 9]

In [59]:
Q =  unlabel_1.loc[pos_to_be_removed,:]

In [60]:
len(Q)

7

In [63]:
import lightgbm as lgb

def classifier_select(train, Q, RN):

  loop_variable =0
  for i in range(0,3):  # comment it change the range 
    print(loop_variable)
    p_and_RN = pd.concat([train,RN])
    p_and_RN.reset_index(inplace=True,drop=True)
    p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
    p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

    Q_vect = tfidf_vect.transform(Q['text'])
    Q_vect_df=pd.DataFrame(Q_vect.toarray())

    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
    np.unique(p_and_RN['Target'])

    Q_pred = lgb_classifier.predict(Q_vect_df)
    np.unique(Q_pred)

    count_q = 0
    total_q = 0 
    out_pos_q = []
    pos_q = []
    for i in Q_pred:
      #print(i)
      if i == -1:
        pos_q.append(count_q)
        total_q = total_q + 1
      else:
        out_pos_q.append(count_q)

      count_q = count_q + 1

    Q.reset_index(inplace=True,drop=True)
    W = Q.loc[pos_q,:]


    if W.empty :
      print("W is empty, came out of loop")
      break
    else:
      Q_new = Q.loc[out_pos_q,:]
      Q = Q_new
      RN = pd.concat([RN,W])
      RN.reset_index(inplace=True,drop=True)
      loop_variable = loop_variable + 1
    print("completed iteration")
  return lgb_classifier

In [64]:
lgb_classifier = classifier_select(train, Q, RN)

0
W is empty, came out of loop


In [65]:
test_pred =  lgb_classifier.predict(t_p.toarray())

In [66]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)

              precision    recall  f1-score   support

           0     0.3922    0.2597    0.3125        77
           1     0.6957    0.6154    0.6531        26
           2     0.6743    0.8839    0.7650       267
           3     0.5263    0.3614    0.4286        83
           4     0.9167    0.3235    0.4783        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6315       521
   macro avg     0.6294    0.4858    0.5256       521
weighted avg     0.6192    0.6315    0.6040       521

